In [3]:
import requests
import json
import time
import math
import numpy as np
from astropy import wcs
from astropy.io import fits
import csv
import sys
from array import array
import os



apik = "sjwxwgavzfpucqaq"
directory = 'C:\\Users\\daniel\\Downloads\\'
file = '10B-1348_det_imagej.fits'
ra = 186.044243327
dec = 8.83186961532
radius = 8
saveinvertedfiles = False
cutoff = 300

dirtofile = directory+file



R1 = requests.post('http://nova.astrometry.net/api/login', data={'request-json': json.dumps({"apikey": apik})})
print("LOGIN:")
print(R1.json(),"\n")



fitsfile = fits.open(dirtofile)
originaldata = fitsfile[0].data
header = fitsfile[0].header

data = -originaldata+np.min(originaldata)+np.max(originaldata)
# data = 100.*(65535./(data-1900.))**1.4
print("simple inversion completed")

filei = file[0:-5]+"_inv"+file[-5:]
dirtofilei = directory+filei

fits.writeto(dirtofilei, data, header, overwrite=True)
print("simple inversion file saved")



R2 = requests.post('http://nova.astrometry.net/api/upload', data={'request-json': json.dumps({"publicly_visible": "y", "allow_modifications": "d", "session": R1.json()["session"], "allow_commercial_use": "d", "crpix_center": True, "scale_units": "degwidth", "scale_lower": 3, "scale_upper": 8, "center_ra": ra, "center_dec": dec, "radius": radius})}, files={'file': open(dirtofilei, 'rb')})
print("\nSUBMISSION:")
print(R2.json(),"\n")



r3strs = 'http://nova.astrometry.net/api/submissions/'
r3strm = str(R2.json()["subid"])
R3 = requests.post(str(r3strs + r3strm))
sys.stdout.write("solving ")
counter = 0

while len(R3.json()["job_calibrations"])==0:
    if (counter>cutoff):
        print("\nSolution failed, taking over",cutoff,"seconds\nExiting program")
        sys.exit()
    time.sleep(1)
    counter += 1
    sys.stdout.write(".")
    R3 = requests.post(str(r3strs + r3strm))

time.sleep(2)
print("\n\nSOLVED:")
print(R3.json())



r4strs = 'http://nova.astrometry.net/api/jobs/'
r4strm = str(R3.json()["jobs"])[1:-1]
r4stre1 = '/objects_in_field/'
r4stre2 = '/calibration/'
R4 = requests.post(str(r4strs+r4strm+r4stre2))
rasol1 = R4.json()["ra"]
decsol1 = R4.json()["dec"]



r5str = 'http://nova.astrometry.net/new_fits_file/' + r4strm
R5 = requests.get(r5str, allow_redirects=True)
sol1ifile = filei[0:-5]+"_sol1"+filei[-5:]
dirtosol1ifile = directory+sol1ifile

open(str(dirtosol1ifile), 'wb').write(R5.content)

print("\ninverted first solution file saved")

fitsfilesol1i = fits.open(dirtosol1ifile)
datasol1i = fitsfilesol1i[0].data
headersol1i = fitsfilesol1i[0].header

datasol1i = originaldata

filesol1 = file[0:-5]+"_sol1"+file[-5:]
dirtosol1file = directory+filesol1

fits.writeto(dirtosol1file, datasol1i, headersol1i, overwrite=True)

print("first solution file saved")

print()
print()
print()



print("SECOND RUN")
R2 = requests.post('http://nova.astrometry.net/api/upload', data={'request-json': json.dumps({"publicly_visible": "y", "allow_modifications": "d", "session": R1.json()["session"], "allow_commercial_use": "d", "crpix_center": True, "scale_units": "degwidth", "scale_lower": 3, "scale_upper": 8, "center_ra": rasol1, "center_dec": decsol1, "radius": radius})}, files={'file': open(dirtosol1ifile, 'rb')})
print("\nSUBMISSION:")
print(R2.json(),"\n")



r3strm = str(R2.json()["subid"])
R3 = requests.post(str(r3strs + r3strm))
sys.stdout.write("solving ")
counter = 0

while len(R3.json()["job_calibrations"])==0:
    if (counter>cutoff):
        print("\nSolution failed, taking over",cutoff,"seconds\nExiting program")
        sys.exit()
    time.sleep(1)
    counter += 1
    sys.stdout.write(".")
    R3 = requests.post(str(r3strs + r3strm))
    
time.sleep(2)
print("\n\nSOLVED:")
print(R3.json())



r4strm = str(R3.json()["jobs"])[1:-1]
R4 = requests.post(str(r4strs+r4strm+r4stre2))
rasol2 = R4.json()["ra"]
decsol2 = R4.json()["dec"]
orisol2 = R4.json()["orientation"]
scalesol2 = R4.json()["pixscale"]
radiussol2 = R4.json()["radius"]
htsol2 = (header['NAXIS2']*scalesol2)/3600.
wdsol2 = (header['NAXIS1']*scalesol2)/3600.



print("\nMETADATA:")
print("RA-FITS:",str(rasol2),"deg")
print("DEC-FITS:",str(decsol2),"deg")
print("SCANHT:",str(htsol2),"deg")
print("SCANWD:",str(wdsol2),"deg")
print("SCANRAD:",str(radiussol2),"deg")
print("SCANSCAL:",str(scalesol2),"arcsec/px")
print("SCAN-ORI: Up is",str(orisol2),"degrees E of N")
print()
print(str(rasol2),"deg"+","+str(decsol2),"deg"+","+str(htsol2),"deg"+","+str(wdsol2),"deg"+","+str(radiussol2),"deg"+","+str(scalesol2),"arcsec/px"+","+"Up is",str(orisol2),"degrees E of N")



r5str = 'http://nova.astrometry.net/new_fits_file/' + r4strm
R5 = requests.get(r5str, allow_redirects=True)
sol2ifile = filei[0:-5]+"_sol2"+filei[-5:]
dirtosol2ifile = directory+sol2ifile

open(str(dirtosol2ifile), 'wb').write(R5.content)

print("\ninverted second solution file saved")

fitsfilesol2i = fits.open(dirtosol2ifile)
datasol2i = fitsfilesol2i[0].data
headersol2i = fitsfilesol2i[0].header

datasol2i = originaldata

filesol2 = file[0:-5]+"_sol2"+file[-5:]
dirtosol2file = directory+filesol2

fits.writeto(dirtosol2file, datasol2i, headersol2i, overwrite=True)

print("second solution file saved")



fitsfilesol2 = fits.open(dirtosol2file)
datasol2 = fitsfilesol2[0].data
headersol2 = fitsfilesol2[0].header
datasol2 = 100.*(65535./(datasol2-1900.))**1.4
print("\ntransformation completed")

filet = filesol2[0:-5]+"_t"+filesol2[-5:]
dirtofilet = directory+filet

fits.writeto(dirtofilet, datasol2, headersol2, overwrite=True)
print("transformation file saved")

fitsfile.close()
fitsfilesol1i.close()
fitsfilesol2i.close()
fitsfilesol2.close()

if (not saveinvertedfiles):
    os.remove(directory+filei)
    time.sleep(1)
    os.remove(directory+sol1ifile)
    time.sleep(1)
    os.remove(directory+sol2ifile)

print("\nPROGRAM COMPLETE")

LOGIN:
{'status': 'success', 'message': 'authenticated user: babnigg@uchicago.edu', 'session': 'bjrlv6s3pxfca4yr514x19msvcbhz0ea'} 



simple inversion completed
simple inversion file saved

SUBMISSION:
{'status': 'success', 'subid': 6130636, 'hash': '2d0a1fa18a7f4de503e7d59c3952d97c72abafe6'} 

solving .........................

SOLVED:
{'user': 36622, 'processing_started': '2022-08-11 19:04:36.698550', 'processing_finished': '2022-08-11 19:04:44.739940', 'user_images': [6268387], 'images': [14722268], 'jobs': [6837888], 'job_calibrations': [[6837888, 4982665]]}

inverted first solution file saved
first solution file saved



SECOND RUN

SUBMISSION:
{'status': 'success', 'subid': 6130638, 'hash': '1d932e338a6993da66097c8a4a9f1962f0ab3be2'} 

solving ..................................

SOLVED:
{'user': 36622, 'processing_started': '2022-08-11 19:05:50.357812', 'processing_finished': '2022-08-11 19:05:57.276207', 'user_images': [6268390], 'images': [14722274], 'jobs': [6837892], 'job_calibrations': [[6837892, 4982671]]}

METADATA:
RA-FITS: 186.55642235699997 deg
DEC-FITS: 8.137636919480002 deg
SCANHT: 6.801039357690325